In [1]:
from functions import *

In [29]:
import matplotlib.pyplot as plt
import glob
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import geopandas as gpd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
import glob
import time
from datetime import datetime

## Load data

### Choose inputs

In [70]:
source_dir = '../data/preprocessed'
data = ['lst','wt','ndvi']
all_ds = {k: [] for k in data}
for d in data:
    source_path = os.path.join(source_dir,d)
    for subdir, dirs, files in os.walk(source_path):
        for file in files:
            if file.endswith('.csv'):
                all_ds[d].append(pd.read_csv(f'{subdir}/{file}',index_col=0))
                

To choose specific rivers or not

In [71]:
filter_river = 0
if filter_river != None:
    #lst_array = all_ds['lst'][filter_river].values.flatten()
    #ndvi_array = all_ds['ndvi'][filter_river].values.flatten()
    #wt_array = all_ds['wt'][filter_river].values.flatten()
    df = all_ds['lst'][filter_river].copy()
    df = df.sort_index(ascending=True)
    indices.extend([idx for idx in df.index for _ in range(df.shape[1])])
    lst_array=df.values.flatten()
            
    df = all_ds['wt'][filter_river].copy()
    df = df.sort_index(ascending=True)
    indices_wt.extend([idx for idx in df.index for _ in range(df.shape[1])])
    wt_array = df.values.flatten()

    if 'ndvi' in data:
        df = all_ds['ndvi'][filter_river].copy()
        df = df.sort_index(ascending=True)
        indices_ndvi.extend([idx for idx in df.index for _ in range(df.shape[1])])
        ndvi_array = df.values.flatten()

else:
    lst_datasets = []
    ndvi_datasets = []
    wt_datasets = []
    indices = []
    indices_ndvi = []
    indices_wt = []
    
    for i in range(len(all_ds['lst'])):
        df = all_ds['lst'][i].copy()
        df = df.sort_index(ascending=True)
        indices.extend([idx for idx in df.index for _ in range(df.shape[1])])
        lst_datasets.append(df.values.flatten())
                
        df = all_ds['wt'][i].copy()
        df = df.sort_index(ascending=True)
        indices_wt.extend([idx for idx in df.index for _ in range(df.shape[1])])
        wt_datasets.append(df.values.flatten())

        if 'ndvi' in data:
            df = all_ds['ndvi'][i].copy()
            df = df.sort_index(ascending=True)
            indices_ndvi.extend([idx for idx in df.index for _ in range(df.shape[1])])
            ndvi_datasets.append(df.values.flatten())

        
    lst_array = np.concatenate(lst_datasets)
    wt_array = np.concatenate(wt_datasets)

    if 'ndvi' in data:
        ndvi_array = np.concatenate(ndvi_datasets)
          
        
    #lst_array = np.concatenate([dataset.values.flatten() for dataset in all_ds['lst']])
    #ndvi_array = np.concatenate([dataset.values.flatten() for dataset in all_ds['ndvi']])
    #wt_array = np.concatenate([dataset.values.flatten() for dataset in all_ds['wt']])
    

In [72]:
print(indices_ndvi==indices_wt)
print(lst_array.shape, ndvi_array.shape, wt_array.shape)

False
(82128,) (82128,) (82128,)


In [73]:
use_ndvi = True
if 'ndvi' in data:
  array_combined = np.stack((lst_array, ndvi_array), axis=-1)
  X = array_combined

else:
  X = lst_array
  X = X.reshape(-1, 1)

y = wt_array

In [74]:
X.shape

(82128, 2)

## Linear Regression model

In [75]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

### K-fold cross validation

In [76]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

model = LinearRegression()

In [77]:
# Iterate over the folds
start_time = time.time()
c=0
all_results=[]
for train_index, test_index in kf.split(X_scaled):
    print('Doing',c)
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Train the model
    model.fit(X_train, y_train)
    print('Entrenat')

    # Predict on the test set
    y_pred = model.predict(X_test)

    print('Calculant mètriques...')
    total_results = evaluate_model(y_test, y_pred)
    all_results.append(total_results)
    
    c+=1

end_time = time.time()

Doing 0
Entrenat
Calculant mètriques...
Doing 1
Entrenat
Calculant mètriques...
Doing 2
Entrenat
Calculant mètriques...
Doing 3
Entrenat
Calculant mètriques...
Doing 4
Entrenat
Calculant mètriques...


In [78]:
mean={}
for i,res in enumerate(all_results):
    for k,v in res.items():
        if i==0:
            mean[k] = [v]
        else:
            mean[k].append(v)

final_mean={}
for k,v in mean.items():
    final_mean[k] = np.mean(v)
    print(k,':',np.mean(v))

MAE : 2.674864344705859
MSE : 22.836099041321056
RMSE : 4.264923527095275
R² : -1.9385735233190537e-05
MAPE (%) : 1303991103479.5251
MSE sample-wise : 18.18974399527686


### Save model results

In [79]:
# Get experiment data
duration = round(end_time - start_time, 2)
current_date = datetime.now().strftime("%Y-%m-%d")
current_time = datetime.now().strftime("%H:%M:%S")

model_name = 'pixel_wise_LR'
# Save model results

details = {'RMSE':final_mean['RMSE'],'Variables':', '.join(data),'Input': f'{len(X)} pixels', 'Output': 'wt', \
           'Split': 'Time-wise','Date':current_date, \
           'Time':current_time, 'Duration': duration}

file_path = f"../results/{model_name}_results.xlsx"
save_excel(file_path, details, excel = 'Results')

final_mean['Model'] = model_name
file_path = f"../results/all_results.xlsx"
save_excel(file_path, final_mean, excel = 'Results')

In [80]:
pd.read_excel(f"../results/{model_name}_results.xlsx")

,RMSE,Variables,Input,Output,Split,Date,Time,Duration
0,3.382903,"lst, wt, ndvi",1128975 pixels,wt,Time-wise,2024-11-01,16:04:59,335.14
1,3.382903,"lst, wt, ndvi",1128975 pixels,wt,Time-wise,2024-11-01,16:05:17,335.14
2,3.384305,"lst, wt",1128975 pixels,wt,Time-wise,2024-11-01,16:12:33,335.47
3,4.264925,"lst, wt",82128 pixels,wt,Time-wise,2024-11-01,16:23:11,24.68
4,4.264924,"lst, wt, ndvi",82128 pixels,wt,Time-wise,2024-11-01,16:26:32,24.59
